# MVP

In [3]:
import pandas as pd
import numpy as np

import logging
import boto3
from botocore.exceptions import ClientError

In [4]:
import MVP_acquire_ds

In [5]:
pd.read_json("df_ds_tx_prepared.json")

,date,title,company,company_rating,job_link,job_description,city,state,zipcode,clean,tokenized,stemmed,lemmatized
0,2021-02-03,Data Scientist Sr. Associate\nnew,"JPMorgan Chase Bank, N.A.",3.9,https://www.indeed.com/rc/clk?jk=dfd1ed6ece073...,The Data Scientist is an individual contributo...,Lewisville,TX,0,data scientist individual contributor able app...,the data scientist is an individual contributo...,the data scientist is an individu contributor ...,the data scientist is an individual contributo...
1,2021-02-03,Data Scientist\nnew,Booz Allen Hamilton,3.9,https://www.indeed.com/rc/clk?jk=246ba9f708c3e...,The Challenge:\nAre you excited at the prospec...,San Antonio,TX,0,challenge excited prospect unlocking secret he...,the challenge\nare you excited at the prospect...,the challeng are you excit at the prospect of ...,the challenge are you excited at the prospect ...
2,2021-02-03,Human Performance Data Analyst Mid/Sr\nnew,Booz Allen Hamilton,3.9,https://www.indeed.com/rc/clk?jk=2538d333374b7...,Key Role:\nServe as a Human Performance Data A...,San Antonio,TX,0,key role serve human performance data analyst ...,key role\nserve as a human performance data an...,key role serv as a human perform data analyst ...,key role serve a a human performance data anal...
3,2021-02-03,Professional-Data Scientist\nnew,AT&T,3.7,https://www.indeed.com/rc/clk?jk=c4841a42f1129...,Overall Purpose: THIS JOB REQUIRES SPECIAL APP...,Plano,TX,0,overall purpose job requires special approval ...,overall purpose this job requires special appr...,overal purpos thi job requir special approv fr...,overall purpose this job requires special appr...
4,2021-02-03,Sr. Data Scientist (Remote)\nnew,Ayasdi,2.3,https://www.indeed.com/rc/clk?jk=57ec78190a3c2...,Ayasdi is breaking new ground in enterprise AI...,Austin,TX,78708,ayasdi breaking new ground enterprise ai looki...,ayasdi is breaking new ground in enterprise ai...,ayasdi is break new ground in enterpris ai and...,ayasdi is breaking new ground in enterprise ai...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,2020-12-22,"Data Scientist - Machine Learning/AI - (US, Re...",Dell Technologies,4.0,https://www.indeed.com/rc/clk?jk=362390b170c4c...,An exciting opportunity for an experienced ind...,Austin,TX,0,exciting opportunity experienced industry prof...,an exciting opportunity for an experienced ind...,an excit opportun for an experienc industri pr...,an exciting opportunity for an experienced ind...
1266,2020-12-22,Data Scientist,JES Tech,0.0,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,The primary function of this position is to ut...,Houston,TX,77058,primary function position utilize data science...,the primary function of this position is to ut...,the primari function of thi posit is to util d...,the primary function of this position is to ut...
1267,2020-12-22,Data Scientist,O'Neil Digital Solutions,3.2,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,JOB SUMMARYDevelops and innovates machine lear...,Plano,TX,75074,job summarydevelops innovates machine learning...,job summarydevelops and innovates machine lear...,job summarydevelop and innov machin learn syst...,job summarydevelops and innovates machine lear...
1268,2020-12-22,"Senior Systems Engineer, SRE, Data and AI",Expedia Group,3.9,https://www.indeed.com/rc/clk?jk=f6768c2a9a973...,"Senior Systems Engineer, SRE, Data and AI\nIf ...",Austin,TX,78758,senior system engineer sre data ai youre right...,senior systems engineer sre data and ai\nif yo...,senior system engin sre data and ai if your th...,senior system engineer sre data and ai if your...


In [6]:
def upload_to_S3_bucket(file_name, bucket='dspreparedjobpostings', object_name=None):
    """
    Upload a file to an S3 bucket
    
    ***Prepared data files must be in JSON format***

    Parameters
    ----------
    file_name: str
        Name of the file to upload.
    
    bucket: str, default="dspreparedjobpostings"
        S3 Bucket the file will be uploaded to.
    
    object_name: str, default=None
        The file name that will appear in AWS S3 bucket.
        If an object_name is not specified, the file will
        have the same name as the file_name
    
    Returns
    -------
    True or False: bool
        True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [7]:
upload_to_S3_bucket("df_ds_tx_prepared.json")

NoCredentialsError: Unable to locate credentials